In [1]:
import sys
sys.path.append("/home/srelins/docs/cy_fdm_builder/src")
from testing_helpers import *
from FDM_builder import *

## TO DO:

* PRIORITY: build observation_period table!!


* start process with dataset
* run dataset build over whole dataset by default - add options to specify table_ids/FDMTable objects

## Testing:

In [ ]:
build_test_environment()

In [ ]:
table_1 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_1",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_1.build()

In [ ]:
table_1.build(event_start_date_cols="date")

In [ ]:
table_1.build(event_end_date_cols="date")

In [2]:
table_2 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_2",
    dataset_id="CY_TESTS_FDM"
)

	 ##### Initialising FDM Table - src_table_2 #####
________________________________________________________________________________

* Copying table src_table_2 to CY_TESTS_FDM - copied!


In [3]:
table_2.build()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_2 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* digest found - syntax correct

2. Adding person_id column:

	* person_id column added!

3. Adding event_start_date:

	No event_start_date column found and event_start_date_cols not specified.

	To complete the build process, an event start date is required. Specify the
	`event_start_date_cols` argument of the .build() method as either:

	* If one column contains the full event date:

	  run .build(event_start_date_cols="insert-column-name-here")
	  NOTE: The process assumes a default date format of year-month-day. If the
	  date column is formatted as day-month-year set the `event_start_date_format`
	  argument as "DMY" (i.e. .build(event_start_date_cols="column-name",
	  event_start_date_format="DMY")) or "MDY" if the format is month-day-year etc....

	* If the year month and day are in sepa

In [4]:
table_2.head()

,person_id,digest,day,month,year
0,24,>J,19,1,2005
1,None,junk_digest_527,26,2,2006
2,23,>I,15,2,2009
3,16,=L,6,2,2003
4,None,junk_digest_659,1,3,2007
5,29,>O,18,3,2028
6,20,>F,28,3,2002
7,22,>H,19,4,2006
8,12,=H,17,4,1985
9,13,=I,27,5,1981


In [5]:
table_2.build(event_start_date_cols=["year", "month", "day"])

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_2 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* digest found - syntax correct

2. Adding person_id column:

	* src_table_2 already contains person_id column

	NOTE: digest also found in src_table_2. If you
	wish to rebuild the person_id column from digest, drop the existing
	person_id column in src_table_2 and run .build() again

3. Adding event_start_date:

	* event_start_date column added!

4. Adding event_end_date:

	No event_end_date column found and event_end_date_column not specified.
	event_end_date end date will be omitted. If you wish to add an event_end_date,
	re-run .build() specifying the `event_end_date` argument.

	Note: Table build process will complete, but the observation periods derived
	from a source table with only an event start date may not be accurate, and 
	may result in erroneous

In [ ]:
table_3 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_3",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_3.build()

In [ ]:
table_3.get_column_names()

In [ ]:
table_3.build(event_start_date_cols=["year", "month", "15"])

In [ ]:
table_4 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_4",
    dataset_id="CY_TESTS_FDM"
)
table_4.build()

In [ ]:
table_4.get_column_names()

In [ ]:
table_4.head()

In [ ]:
table_4.build(event_start_date_cols="date_string",
              event_start_date_format="DMY")

In [ ]:
table_4.head()

In [ ]:
table_5 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_5"
    
)